<a href="https://colab.research.google.com/github/yastika/MscFE_Capstone/blob/main/Climate_FinTech_Solution_Internalizing_Environmental_Risks_into_Financial_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Section 1: Introduction**

# **Climate-Adjusted Credit Scoring Prototype**
This notebook demonstrates how to internalize environmental risks (like floods and heatwaves) into credit scoring for SMEs using Gradient Boosting Machine (GBM) and a prototype Streamlit dashboard.


**Section 2: Data Collection and Integration**

## **Step 1: Data Collection and Integration**
We merge SME-level financial data and regional climate event history based on region and year.


In [ ]:
import pandas as pd

# Load datasets
climate_df = pd.read_csv("flood_heatwave_data.csv")
sme_financial_df = pd.read_csv("sme_financials.csv")

# Merge on region and year
combined_df = pd.merge(climate_df, sme_financial_df, on=["region", "year"])
combined_df.head()

**Section 3: Feature Engineering**

## **Step 2: Feature Engineering**
We construct normalized environmental scores and select relevant financial features.


In [ ]:
baseline_temp = 30  # Hypothetical baseline

# Normalize climate risk features
combined_df["flood_risk"] = combined_df["flood_frequency"] / combined_df["flood_frequency"].max()
combined_df["heat_index"] = (combined_df["avg_temp"] - baseline_temp) / 10

# Select modeling features
features = combined_df[["flood_risk", "heat_index", "credit_score", "debt_ratio"]]
target = combined_df["defaulted"]


**Section 4: Model Development**

## **Step 3: Model Training (Gradient Boosting Classifier)**
Train a GBM model using SME and climate features to predict default probabilities.


In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, mean_squared_error

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Train model
model = GradientBoostingClassifier()
model.fit(X_train, y_train)

# Predictions
predictions = model.predict_proba(X_test)[:, 1]

# Evaluation metrics
auc = roc_auc_score(y_test, predictions)
rmse = mean_squared_error(y_test, predictions, squared=False)

print(f"AUC: {auc:.2f}")
print(f"RMSE: {rmse:.2f}")

**Section 5: Climate Risk Scoring Function**

## **Step 4: Climate Credit Risk Score Labeling**
Convert numeric probability scores into categorical labels: Low, Medium, High Risk.


In [ ]:
def climate_credit_risk_score(probability):
    if probability > 0.7:
        return "High Risk"
    elif probability > 0.4:
        return "Medium Risk"
    else:
        return "Low Risk"

# Example usage
example_prob = 0.65
print(f"Risk Category: {climate_credit_risk_score(example_prob)}")

## **Step 5: Prototype Dashboard**

**Interactive Credit Scoring Dashboard with Ipywidgets**

In [ ]:
import ipywidgets as widgets
from IPython.display import display, Markdown

# Define the interactive widgets
flood_slider = widgets.FloatSlider(value=0.3, min=0.0, max=1.0, step=0.01, description='Flood Risk')
heat_slider = widgets.FloatSlider(value=0.2, min=0.0, max=1.0, step=0.01, description='Heat Index')
credit_slider = widgets.IntSlider(value=600, min=300, max=850, step=10, description='Credit Score')
debt_slider = widgets.FloatSlider(value=0.5, min=0.0, max=1.0, step=0.01, description='Debt Ratio')

# Define function to run model and display result
def update_risk(flood, heat, credit_score, debt_ratio):
    input_vector = [[flood, heat, credit_score, debt_ratio]]
    prob = model.predict_proba(input_vector)[0][1]
    risk_label = climate_credit_risk_score(prob)

    display(Markdown(f"### 📊 Predicted Risk Probability: `{prob:.2f}`"))
    display(Markdown(f"### 🛡️ Risk Category: `{risk_label}`"))

# Display interactive widget panel
widgets.interact(update_risk,
                 flood=flood_slider,
                 heat=heat_slider,
                 credit_score=credit_slider,
                 debt_ratio=debt_slider)